In [1]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd
from collections import Counter
import itertools

# NLP用ライブラリ
import MeCab,re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# CSVの読み込み
wikiData = pd.read_csv("csv/train.csv", encoding="utf-8")

# ストップワードの読み込み
path = "data/Slothlib.txt"
stopWords = []
with open(path, encoding="utf-8") as f:
    stopWords = [s.strip() for s in f.readlines()]

# 読み込みデータの表示
wikiData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー（Dole Food Compan...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場（みなみがおかぼくじょう）は、栃木県那須高原に本拠を置く...,1
2,岩瀬牧場 岩瀬牧場（いわせぼくじょう）は、日本の福島県岩瀬郡鏡石町にある牧場である。 ...,1
3,"クラギ クラギ株式会社（くらぎ、""KURAGI Co.Ltd."" ）は、三重県を中心に「...",1
4,"ハッピーネモファーム 株式会社ハッピーネモファーム（""""）は、北海道浦河郡浦河町にある競...",1


In [3]:
# 形態素解析用関数
def tokenize(text,wordClassList):
    tokens = []
    tagger = MeCab.Tagger()
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] in wordClassList :
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [4]:
cleanedData = wikiData.copy()

# 数字の除外
cleanedData['text'] = cleanedData['text'].str.replace('[0-9]+','', regex=True)

# 不要な記号の除外
path = "data/Removal Symbols.txt"
removal_symbols = []
with open(path, encoding="utf-8") as f:
    removal_symbols = [s.strip() for s in f.readlines()]
print(removal_symbols)

for symbol in removal_symbols:
    cleanedData['text'] = cleanedData['text'].str.replace(symbol,'')

['(', ')', '（', '）', '"', '""', '「', '」', '[', ']', '', '', '']


In [5]:
cleanedData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニーDole Food Company...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場みなみがおかぼくじょうは、栃木県那須高原に本拠を置く酪農...,1
2,岩瀬牧場 岩瀬牧場いわせぼくじょうは、日本の福島県岩瀬郡鏡石町にある牧場である。 鏡石...,1
3,クラギ クラギ株式会社くらぎ、KURAGI Co.Ltd. は、三重県を中心に農業屋など...,1
4,ハッピーネモファーム 株式会社ハッピーネモファームは、北海道浦河郡浦河町にある競走馬サラ...,1


In [6]:
# Numpy配列へ変換
X = cleanedData.text.values
y = cleanedData.target.values

In [7]:
# 形態素解析
wordClassList = ['名詞']
words = []
for text in X:
    words.append(tokenize(text,wordClassList))
    
print(len(words))

11460


In [8]:
# 1次元配列へ変換
words_1d = itertools.chain.from_iterable(words)

In [9]:
# 単語別の登場回数をカウント
delWord = []
counter = Counter(words_1d)
for word, count in counter.most_common():
    if count == 1:
        delWord.append(word)
    print(f"{word}: {count}")

年: 39483
年月: 18030
年月日: 15521
こと: 14626
株式会社: 11036
事業: 10892
本社: 10841
店舗: 10236
販売: 9716
店: 8096
製造: 8033
現在: 7959
企業: 7809
ため: 6965
設立: 6884
平成: 6788
円: 5840
日本: 5836
会社: 5813
昭和: 5431
展開: 5365
後: 5041
開発: 4729
運営: 4578
社: 4415
用: 4297
等: 4234
グループ: 4217
メーカー: 4177
製品: 4155
同社: 4134
変更: 4097
中心: 4095
開始: 4043
化: 3893
子会社: 3809
もの: 3697
商品: 3574
使用: 3572
億: 3541
ブランド: 3537
工場: 3491
よう: 3453
社名: 3443
これ: 3431
合併: 3417
的: 3381
経営: 3274
その後: 3265
サービス: 3242
創業: 3149
の: 3044
年平: 3026
当時: 3015
成年: 3005
放送: 2968
路線: 2951
営業: 2936
運行: 2886
主: 2851
内: 2829
向け: 2807
発表: 2802
一部: 2775
月日: 2740
時: 2718
位: 2710
ほか: 2668
中: 2599
万: 2586
社長: 2541
他: 2513
導入: 2464
発売: 2376
生産: 2324
以降: 2314
提供: 2286
部門: 2265
年代: 2208
車両: 2204
業務: 2194
名: 2187
利用: 2167
株式: 2165
バス: 2145
当初: 2138
線: 2131
存在: 2090
出店: 2036
CM: 2015
間: 1954
東京: 1948
システム: 1923
車: 1914
番組: 1909
関連: 1908
数: 1890
期: 1887
傘下: 1867
機器: 1849
系: 1844
関係: 1814
上: 1811
目: 1757
者: 1754
人: 1748
統合: 1735
地域: 1729
階: 1729
月: 1712
国内: 1710
前: 1650
営

システムインテグレーター: 131
日本全国: 130
言葉: 130
青森県: 130
本数: 130
経済: 130
自動車メーカー: 130
リサイクル: 130
末期: 130
五: 130
製薬会社: 130
西武百貨店: 130
愛媛: 130
フランチャイジー: 130
調剤薬局: 130
ハイビジョン: 130
敗退: 130
そば: 129
一定: 129
世紀: 129
合同: 129
債務: 129
ユニット: 129
一体: 129
マスコットキャラクター: 129
人事: 129
コラボレーション: 129
最高: 129
中核企業: 129
乗降: 129
ペット: 128
プロ野球: 128
清酒: 128
製法: 128
出典: 128
ターゲット: 128
PR: 128
情報システム: 128
アプリケーション: 128
自社制作: 128
バス路線: 128
お客様: 127
兆: 127
主導: 127
物件: 127
豊富: 127
本業: 127
札幌市: 127
期間限定: 127
惣菜: 127
太陽: 127
刊行: 127
専門商社: 127
チーム名: 127
店員: 127
事情: 127
ドライバー: 127
Web: 127
現存: 126
三井: 126
なか: 126
返済: 126
改革: 126
提示: 126
降格: 126
総額: 126
最新: 126
ストア: 126
権利: 126
午後: 126
屋上: 126
聴取: 126
プラザ: 126
送信所: 126
改名: 125
宮崎: 125
東証一部: 125
現場: 125
三洋電機: 125
アナウンス: 125
機関: 125
行為: 125
com: 125
パチンコ: 125
大人: 125
光学: 125
HC: 125
店舗面積: 125
ヘッドコーチ: 125
業種: 124
防止: 124
コーポレーション: 124
距離: 124
プラン: 124
心: 124
禁止: 124
たち: 124
カバー: 124
青: 124
ショッピング: 124
買い物: 124
開幕戦: 124
オンキヨー: 124
ch: 124
直通: 124
農業: 123
野菜: 123
インドネシア: 123
創造: 123
飲食:

補償: 69
神奈川県横浜市: 69
急増: 69
肥料: 69
弱: 69
ブロック: 69
実態: 69
現代: 69
製パン: 69
オプション: 69
マーケット: 69
新ブランド: 69
記号: 69
始まり: 69
日本プロサッカーリーグ: 69
羽田: 69
姉妹: 69
社内カンパニー: 69
特性: 69
平和: 69
製紙: 69
到着: 69
キャピタル: 69
前年比: 69
財界: 69
ミュージック: 69
ケーズデンキ: 69
角川書店: 69
タクシー会社: 69
冬: 68
食品事業: 68
タンク: 68
発信: 68
小: 68
市街地: 68
不適切: 68
日本企業: 68
サービス開始: 68
味の素: 68
スープ: 68
君: 68
ファイナンス: 68
国際線: 68
社会人: 68
具: 68
財閥: 68
合名会社: 68
クリスタル: 68
車椅子: 68
スタンプ: 68
脱退: 68
行先: 68
半額: 68
方向幕: 68
MHz: 68
鉄道模型: 68
都市銀行: 68
生放送: 68
北海道中央: 68
フィリピン: 67
衰退: 67
殆ど: 67
日本陶器: 67
生産能力: 67
名古屋証券取引所: 67
香川県高松市: 67
海外事業: 67
ビジネスモデル: 67
東京支店: 67
大阪市中央: 67
連結決算: 67
救済: 67
都心: 67
都市型: 67
簡易: 67
組み合わせ: 67
催事: 67
日付: 67
定番: 67
ハイ: 67
芸人: 67
好き: 67
製糖: 67
打撃: 67
発売元: 67
JASDAQ: 67
レギュラー: 67
祝日: 67
連動: 67
GM: 67
元気: 67
林: 67
ロッテ: 67
容易: 67
段ボール: 67
送電: 67
高性能: 67
AC: 67
産業機械: 67
BMW: 67
盤: 67
私鉄: 67
レーシング: 67
スマホ: 67
中位: 67
平和堂: 67
中心市街地: 67
ハイデッカー: 67
難: 66
新: 66
不祥事: 66
馬: 66
魚: 66
数々: 66
京急: 66
しま: 66
道東: 66
コールセンター: 66
シフト: 66
娘: 66
NTT東日本: 66
景気: 6

大輪会: 45
工法: 45
在来線: 45
産業廃棄物: 45
メガ: 45
右衛門: 45
決意: 45
兵庫県姫路市: 45
一円: 45
フロンティア: 45
川崎市: 45
アカウント: 45
コマツ: 45
学: 45
開校: 45
Service: 45
日本市場: 45
小麦: 45
漢字: 45
綿: 45
感謝: 45
佐々木: 45
ぞう: 45
海外向け: 45
樽: 45
医薬部外品: 45
玉: 45
コカ・コーラウエスト: 45
陸軍: 45
勢力: 45
ナイター: 45
地元企業: 45
連結子会社化: 45
豆: 45
レストランチェーン: 45
投資家: 45
西側: 45
観客: 45
カテゴリ: 45
抵抗: 45
最適: 45
ベンチャー: 45
設備投資: 45
しながら: 45
サーバー: 45
千歳: 45
許諾: 45
讃岐: 45
パフォーマンス: 45
年生: 45
in: 45
実店舗: 45
躍進: 45
専務取締役: 45
アルバム: 45
長野市: 45
トップバリュ: 45
アパレルメーカー: 45
出雲: 45
指名: 45
サーキット: 45
注入: 45
飛行機: 45
ラリー: 45
メルク: 45
初日: 45
新田: 45
セルフ: 45
セラミック: 45
超音波: 45
ニッケル: 45
上り: 45
ワンステップバス: 45
カラオケボックス: 45
琉球: 45
LDH: 45
さくら銀行: 45
配車: 45
準決勝: 45
スイーツ: 44
最終赤字: 44
蒲鉾: 44
鉄道路線: 44
終止符: 44
炭: 44
成東: 44
新潟市: 44
すみ: 44
良好: 44
三水会: 44
罰金: 44
ヤマト: 44
とうほ: 44
いち: 44
新日本製鐵: 44
東海旅客鉄道: 44
特別損失: 44
掲出: 44
身: 44
家賃: 44
暮らし: 44
疑問: 44
特集: 44
山一證券: 44
サブ: 44
チャレンジ: 44
天皇: 44
一線: 44
書体: 44
医薬: 44
PB: 44
若者: 44
冒頭: 44
戦争: 44
粉: 44
北陸鉄道: 44
MBSラジオ: 44
義務: 44
追随: 44
昭和初期: 44
試薬: 44
TBS系列: 4

副会長: 33
分裂: 33
AFCチャンピオンズリーグ: 33
下関: 33
福岡ソフトバンクホークス: 33
仕込み: 33
ビスケット: 33
幼児: 33
土産: 33
少量: 33
弾力: 33
メロディ: 33
明治製菓: 33
しょうゆ: 33
CBCラジオ: 33
各県: 33
元禄: 33
鳥取県鳥取市: 33
MBS: 33
富士山: 33
新鮮: 33
マスメディア: 33
NIPPON: 33
別々: 33
クリーム: 33
客層: 33
保育園: 33
厳密: 33
小文字: 33
延: 33
ハウステンボス: 33
出て: 33
契約期間: 33
市長: 33
仮称: 33
歴: 33
庵: 33
イースト: 33
with: 33
出品: 33
一種: 33
キャラクターグッズ: 33
斡旋: 33
大塚: 33
オイル: 33
大阪府内: 33
サッポロ: 33
津: 33
農家: 33
残: 33
社債: 33
チャネル: 33
ボディー: 33
ハート: 33
西川: 33
セブン: 33
寄贈: 33
販売拠点: 33
日本製紙: 33
リサーチ: 33
技研: 33
階段: 33
長崎市: 33
JWAVE: 33
最大限: 33
図案: 33
医師: 33
水力発電: 33
芸能人: 33
帝人: 33
三菱金曜会: 33
トレーラー: 33
ソウル: 33
庄内: 33
兼用: 33
電器: 33
倉敷: 33
淀川: 33
日本軽金属: 33
ホールディング: 33
製本: 33
チタン: 33
貴金属: 33
納期: 33
中間決算: 33
洗濯: 33
高炉メーカー: 33
フジサンケイグループ: 33
年内: 33
レール: 33
燃焼: 33
エアバス: 33
デンソー: 33
日本冶金工業: 33
サスペンション: 33
総会: 33
ディーゼル: 33
キーボード: 33
制御装置: 33
パーソナルコンピュータ: 33
WD: 33
リフト: 33
コム: 33
虎彦: 33
クリエイター: 33
電化製品: 33
ルネサス: 33
DCG: 33
図: 33
ダイハツ工業: 33
ドック: 33
阿部: 33
土佐電気鉄道: 33
なかよし: 33
貸出: 33
フットボール: 33
デビットカード: 3

...

フリータイム: 1
カラコロッタ: 1
アニマロッタ: 1
ツナガロッタ: 1
グランデザイヤー: 1
マーブルフィーバー: 1
スーパーミリオンビンゴ: 1
カラオケ・ダーツ・ビリヤード・スポッチャレジャスタ: 1
つき日: 1
ラウポケアプリ: 1
サントロペ: 1
風俗営業法: 1
ワイビーエス: 1
リズ: 1
GlobalSoftware: 1
輝昭: 1
広島県総合グランド: 1
株式会社 ニットー: 1
広島県公安委員会: 1
広島信用金庫: 1
development: 1
大分県中津市東本町: 1
グランプラザ中津ホテル: 1
株式会社進学会: 1
がくり: 1
ょくぞうしんかいの: 1
平井睦雄: 1
崇浩: 1
教学図書協会: 1
学歴: 1
歳児: 1
東大学力増進会: 1
福島ユナイテッドフットボールクラブ: 1
十六沼公園: 1
あづま総合運動公園: 1
エフエム福島: 1
JUNKERS: 1
福島県社会人サッカーリーグ: 1
時崎悠: 1
盛岡ゼブラ: 1
FCシャイネン福島: 1
AC福島ユナイテッド: 1
柳原裕: 1
大原卓丈: 1
星雄次: 1
金弘淵: 1
小牟田洋佑: 1
福島県サッカー選手権大会: 1
株式会社東京會舘: 1
偲ぶ会: 1
プルニエ: 1
シェ・ロッシニ: 1
東苑: 1
二重橋前: 1
日比谷駅: 1
田辺淳吉: 1
大東亜: 1
丸の内二重橋ビル: 1
スカイラウンジ: 1
トリトンスクエア: 1
コヤマ: 1
鳥居本宿: 1
サンライズスタディオ: 1
土橋町: 1
鳥居本: 1
DUET: 1
淡海文化を育てる会: 1
レーク: 1
もへろん: 1
たひ: 1
無料情報誌: 1
ローソンステーションカード: 1
LEncore: 1
オフィシャルチケットステーション: 1
野林: 1
定行: 1
日比: 1
靖浩: 1
野林徳行: 1
株式会社九九プラス: 1
椿事: 1
株式会社ぱど: 1
PADO Corporation: 1
check: 1
タブロイド版: 1
国勢調査: 1
よう約: 1
Power M: 1
靭本: 1
ぼうはん: 1
菊川凱夫: 1
藤枝ブルックス: 1
中央防犯サッカー部: 1
Zweigen: 1
zwei: 1
ゲーエン: 1
gehen: 1
金沢弁: 1
ク

永井大: 1
松田悟志: 1
臼田あさ美: 1
秋田県民: 1
株式会社エヌ・ティ・ティ・データ経営研究所: 1
きゅうじょ: 1
QUNIE: 1
株式会社夢テクノロジー: 1
理科系: 1
フルキャストファクトリー: 1
フルキャストセントラル: 1
管理責任者: 1
フリーワーク: 1
ワセ: 1
アカ: 1
須野田誠: 1
開成高校: 1
医歯薬専門予備校: 1
学館: 1
須野田珠美: 1
英進館株式会社: 1
予習シリーズ: 1
NN: 1
苗場: 1
無施錠: 1
長野県警: 1
中野署: 1
落ち度: 1
建造物侵入: 1
株式会社アセットマネジメント: 1
スカイジャルスカイ: 1
ウェイブ: 1
グランドスタッフ: 1
ADOArea: 1
フィーノ: 1
Fino: 1
ホテルフィーノ: 1
ベストウエスタンブランド: 1
クリード: 1
カンデオホテルズ: 1
ベストウェスタンホテルブランド: 1
イエヒトイエヒト: 1
建築科: 1
品確法: 1
浸入: 1
資力: 1
カードホテルシステム: 1
JR九州エンジニアリング株式会社: 1
ケイ・エス・ケイ: 1
サン・クエスト株式会社: 1
メリィメリィ: 1
名人戦: 1
トビタテ留学JAPAN: 1
東山動物園: 1
秋まつり: 1
佐藤英夫: 1
佐藤塾: 1
StarBoard: 1
個別学習: 1
古藤: 1
Comets: 1
コメッツ: 1
Will: 1
新小: 1
佐鳴湖: 1
さなるこ: 1
三島進学ゼミナール: 1
サナスタ: 1
初等部: 1
HighSchool: 1
７つ: 1
小４: 1
ド根性: 1
校訓: 1
キッズジャーナル: 1
中園化学: 1
中園化学株式会社: 1
大分日田: 1
タナベ経営: 1
株式会社タナベ経営: 1
TANABE MANAGEMENT CONSULTING COLTD: 1
田辺昇一: 1
オールラウンド: 1
ＲＥＶＩＥＷ: 1
人材マネジメント: 1
戦略財務: 1
幹部候補生: 1
セミナーチームリーダースクール: 1
ＣＤ: 1
ＰＲ: 1
デザインツールプレミアムノベルティ: 1
ブルーダイアリーダイアリー: 1
ダイアリー: 1
ジャパンエリアコードTV株式会社: 1
ジャパンエリアコードティービー: 1
小栗徳丸

エヌケーメディコ: 1
活性酸素: 1
修飾: 1
変性: 1
掛け算: 1
採尿: 1
エルファブ: 1
アルブミン: 1
実年齢: 1
株式会社日本医療データセンター: 1
JMDC: 1
血管年齢: 1
免疫力: 1
マイクロアレイ: 1
特異度: 1
Cologic: 1
コロジック: 1
すい臓癌: 1
PanaSee: 1
パナシー: 1
ミアテスト: 1
マイクロRNA: 1
遺伝子解析: 1
アミロイドβ: 1
IGA: 1
アイジーエー: 1
くのいち: 1
BUNKER: 1
大阪市西区南堀江: 1
ゴスペル: 1
大西忠司: 1
佐藤美佳子: 1
高木愛美: 1
彰彦: 1
有恒: 1
トヨタ博物館: 1
みれにあむ: 1
オベッカ: 1
ブリューナクケルト: 1
船浦: 1
舞田: 1
浦安市運動公園陸上競技場: 1
株式会社キタック: 1
KITAC CORPORATION: 1
土質: 1
環境分析: 1
ディー・エヌ・エー: 1
ウェルク: 1
和博: 1
祐司: 1
イエモ: 1
メリー: 1
鈴木裕斗: 1
オールアバウト: 1
久輝: 1
Buzz: 1
バズニュース: 1
バイラル: 1
まとめ記事: 1
守安功: 1
エビデンス: 1
正浩: 1
転職サイト: 1
自己分析: 1
幽霊: 1
音喜多駿: 1
東京都福祉保健局: 1
アトピー性皮膚炎: 1
cuta: 1
UpIn: 1
CAFY: 1
JOOY: 1
GOIN: 1
PUUL: 1
リクルートホールディングス: 1
乱造: 1
スパムサイト: 1
南場: 1
クラウドワークス: 1
ヨッピー: 1
大炎上: 1
複製権: 1
免罪符: 1
ペロリ: 1
現場責任者: 1
Meridian: 1
牛窪万里子: 1
メリディアンアナウンススクール: 1
静岡県大手: 1
静岡県浜松市北区: 1
引佐町: 1
活性汚泥: 1
天日: 1
木くず: 1
燃え殻: 1
FIVE: 1
株式会社加賀: 1
與吉郎: 1
陶芸家: 1
年雄: 1
生活科: 1
北投区: 1
北投温泉: 1
勝生: 1
辻口博啓: 1
YUKIZURI: 1
ユキズリ: 1
辻口: 1
ミュゼ: 1
アッシュカフェ: 1
千葉県鋸南町: 1
ジェイピィールーム: 1
寺井禎浩: 1
真野恵里

ODH: 1
本谷紗己: 1
TGC: 1
cawaii: 1
麻里: 1
井岡一翔: 1
彩香: 1
トップライバー: 1
田中伸明: 1
華道: 1
IKENOBOYS: 1
ミュージックステーション: 1
乃木: 1
ヴェルサイユ宮殿: 1
アイドル活動: 1
idol: 1
新選組: 1
チームA: 1
Mモデル: 1
Alldemade: 1
Rag: 1
Doll: 1
ラグドール: 1
渡辺千尋: 1
Candy: 1
キャンディココ: 1
利奈: 1
スペード: 1
口付け: 1
pick: 1
カルミア: 1
QUESTION: 1
クエスチョン: 1
TAKA: 1
スパークリングワイン: 1
NEUF: 1
長島観光開発株式会社: 1
ナガシマリゾートブランド: 1
株式会社イード: 1
IID Inc: 1
Response: 1
マイナビ出版: 1
marble: 1
Enth: 1
鷹松宏一: 1
宏一: 1
AGN: 1
株式会社エービー: 1
サンルートホテルチェーン: 1
Bright: 1
DEARSTAGE: 1
でんぱ組inc: 1
ベボガ: 1
愛乙女☆DOLL: 1
Chu☆OhDolly: 1
永田寛哲: 1
アクアルナ・エンターテイメント: 1
サコス: 1
サコス株式会社: 1
SACOS CORPORATION: 1
京王フェアウェルサポート株式会社: 1
グリフィン・レーシングチーム: 1
Gunma: 1
Grifin: 1
グリフィン: 1
Ventos: 1
VENTO: 1
群馬大会: 1
ツーリストコーポレートビジネス: 1
近畿日本ツーリスト株式会社: 1
プレッソイン: 1
TechnoPro Holdings: 1
プロンプトホールディングス: 1
メイテック: 1
ジュリアナ東京: 1
ヴェルファーレ: 1
グッドウィル・インダストリーズ: 1
コミュニティ・ビジネス: 1
防衛庁跡地: 1
ラディアスマイル: 1
一彰: 1
プロンプトホールディングス株式会社: 1
テクノプロ・エンジニアリング: 1
株式会社CSI: 1
株式会社ハイテック: 1
不利益処分: 1
JR長野鉄道サービス株式会社: 1
SHiDAX: 1
太平洋ベルト: 1
シダックスビレッジクラブ: 1
厳罰化: 1
サンバ

中川洋介: 1
小泉勇人: 1
長谷部茂利: 1
笠原昂史: 1
林陵平: 1
木下高彰: 1
ジョインビレEC: 1
岸本武流: 1
バイアーノ: 1
黒川淳史: 1
長谷川凌: 1
市立船橋: 1
平野佑一: 1
国士舘大: 1
大原彰輝: 1
全国高等学校総合体育大会: 1
岡田広: 1
水戸商業高校: 1
犬猿の仲: 1
禍根: 1
JWAY: 1
寺田忍: 1
サポーター席: 1
水戸納豆: 1
ねばり: 1
水戸市水府町: 1
水戸市立サッカー・ラグビー場: 1
城里: 1
アツマーレ: 1
公営住宅: 1
水戸市民: 1
加藤浩一: 1
市側: 1
浦和サポーター: 1
専守防衛: 1
リアクション: 1
自陣: 1
波状攻撃: 1
リトリート: 1
望月一仁: 1
カテナチオ: 1
サッカーファン: 1
ラインダンス: 1
チアリーダー: 1
水戸市民体育館: 1
蝶野正洋: 1
am: 1
Chono: 1
人台: 1
イデラキャピタルマネジメント: 1
株式会社イデラキャピタルマネジメント: 1
手間いらず株式会社: 1
TEMAIRAZU: 1
グローバルトラベルオンライン: 1
ネクスト株式会社: 1
ねくすと: 1
MottoNextInc: 1
那須与一: 1
屋島の戦い: 1
チーム練習: 1
高松市牟礼総合体育館: 1
香川県立体育館: 1
ジュリアス・アシュビー: 1
なみはやドーム: 1
ワイルドカードゲーム: 1
シャークス: 1
宮本真司: 1
城間修平: 1
マット・ギャリソン: 1
井上聡: 1
大阪府立体育会館: 1
じん帯: 1
守口市民体育館: 1
カール・ジョン・ニューマン: 1
野々口: 1
長尾強司: 1
高松市香川総合体育館: 1
仙台市体育館: 1
アントワン・ブロキシー: 1
テランス・ファーリー: 1
啓士朗: 1
ワキ・ウィリアムス: 1
一時帰国: 1
松田大地: 1
京都府立体育館: 1
星島郁洋: 1
那覇市民体育館: 1
ファウルトラブル: 1
気迫: 1
レモンズ: 1
ニカ・ウィリアムスパシフィック: 1
ケヴィン・スミス: 1
リッチモンド: 1
ドイツリーグ: 1
マイケル・ナナリー: 1
早水: 1
ロバート・ジョンソン: 1
ブランドン・ペン: 1
リーグ制覇: 1
米澤翼: 1


In [10]:
# 登場回数の低い単語を削除
#for dw in delWord:
    #for i in range(len(words)): 
        #words[i] = [word for word in words[i] if word != dw]

In [11]:
# ストップワードの読み込み
path = "data/Slothlib.txt"
stopWords = []
with open(path, encoding="utf-8") as f:
    stopWords = [s.strip() for s in f.readlines()]
    
for sw in stopWords:
    for i in range(len(words)): 
        words[i] = [word for word in words[i] if word != sw]   

In [12]:
# 分かち書きを行ったテキストへ変換
cleanedData['text'] = list(map(lambda text:str(text) , words))
cleanedData['text'] = cleanedData['text'].str.replace(","," ")
cleanedData['text'] = cleanedData['text'].str.replace("'","")
cleanedData['text'] = cleanedData['text'].str.replace("[","")
cleanedData['text'] = cleanedData['text'].str.replace("]","")

In [13]:
# CSV出力
cleanedData.to_csv("csv/train_cleaned.csv",index = False, encoding='utf-8')